# Chapter 1, Just basic OCaml

The chapter is just basic OCaml, types and stuff...

### Numbers

In [6]:
(* Integers *)
3 + 8 ;;
10_000 + 20_000 ;; (* Long separators *)
0x12af ;; (* Hex integers *)
0o107 ;; (* Octal numbers *)
0b0010 ;; (* Binary integers *)

- : int = 11


- : int = 30000


- : int = 4783


- : int = 71


- : int = 2


In [9]:
(* Floats *)
1. +. 0.43 ;; (* Note the +. operator, it is needed for floats *)
1e-5 *. 0.5

- : float = 1.43


- : float = 5e-06


### Strings and chars

In [42]:
"hello" ;;
'c' (* this is a char *)

- : string = "hello"


- : char = c


### Variables

In [10]:
let x = 5 + 6

val x : int = 11


Variable names _need to_ start with lowercase

### Functions

They are declared like variables but with the parameters:

In [11]:
let sum a b = a + b

val sum : int -> int -> int = <fun>


Notice the compiler infers the types for the function, this is the real strength of this language, you don't have to "tell it" what type it is, it will know, if you pass or expect something else you are probably wrong. Sometimes it will mean it is a _generic_ type because it could be anything.

We can pass a function as parater too.

Notice the signature of the following function:

In [12]:
let first_if_true test a b =
  if test a then a else b

val first_if_true : ('a -> bool) -> 'a -> 'a -> 'a = <fun>


### Tuples and Lists

Tuples are immutable and declared between parenthesis separated by colon. The type is defined as `'a * 'b`

In [13]:
let a = (4,5)

val a : int * int = (4, 5)


Lists are between square brackets (`[`) and separated by semicolon (`;`)

In [17]:
let b = [4;5]

val b : int list = [4; 5]


You can create lists joining with the operator `append` (`::`), you have to add in this case an empty list at the end.

In [20]:
'a' :: 'b' :: 'c' :: []

- : char list = [a; b; c]


### Pattern matching

This is the hot thing, it basically work with any type, including lists and tuples...

In [15]:
let x, y = a

val x : int = 4
val y : int = 5


With lists it works a little different, we can match against invididual elements in the list but when our pattern is not "comprehensive" (that means, it won't cover all the cases), it will warn us.

In [23]:
let preferred_language languages =
  match languages with
  | first :: the_rest -> first
  | _ -> "OCaml"
;;

preferred_language ["Go";"C#"] ;;
preferred_language []

val preferred_language : string list -> string = <fun>


- : string = "Go"


- : string = "OCaml"


### Recursive functions

We have to decorate a recursive function with `rec`

In [26]:
let rec destutter l =
  match l with
  | [] -> []
  | [a] -> [a]
  | a :: b :: tl ->
    if a = b then destutter (b :: tl) else
    a :: destutter (b :: tl)
;;

val destutter : 'a list -> 'a list = <fun>


Notice the usage of list pattern matching with recursive functions, that is basically your day to day in OCaml.

### Optional

The type `optional` basically means "you have a value, indicated by `Some` or you don't, indicate by `None`"

In [35]:
let log_entry maybe_value message =
  let value = 
    match maybe_value with
    | Some x -> x
    | None -> "None" in
  "Value is " ^ value
;;

val log_entry : string option -> 'a -> string = <fun>


Notice the usage of `match` to find when we have or not a value, the usage of `in` to inline our variable declaration and value in the operation and the usage of the operator `^`.

### Records and variants (and more pattern matching)

We can create our own types, easily:

In [36]:
type point2d = {
  x: float;
  y: float
} ;;

type point2d = { x : float; y : float; }


We can do _field punning_ as well

In [39]:
let magnitude {x; y} = sqrt( x ** 2. +. y ** 2.)

val magnitude : point2d -> float = <fun>


That defines a function using a type point2d (yeah, I know it takes a while to understand!).

How does the compiler knows it is a point2d and not something else? well, because the function uses two members and both members are float (notice the `+.` operator).

We can accept different parameters or use them to define higher order types

In [44]:
type circle_desc = { radius: float; center: point2d };;
type rect_desc = { lower_left: point2d; height: float; width: float };;
type segment_desc = { endpoint1: point2d; endpoint2: point2d };;

type scene_element =
  | Circle of circle_desc
  | Rect of rect_desc
  | Segment of segment_desc
;;

type circle_desc = { radius : float; center : point2d; }


type rect_desc = { lower_left : point2d; height : float; width : float; }


type segment_desc = { endpoint1 : point2d; endpoint2 : point2d; }


type scene_element =
    Circle of circle_desc
  | Rect of rect_desc
  | Segment of segment_desc


## Mutable structures

So far all the structures (tuples, lists, records) are immutable. _Sometimes_ you need mutable structures, or _return absolutely nothing_.

### Arrays

Arrays are mutable, adding, removing, appending, modifying elements of an array is a constant operation (compared to lists).

In [3]:
let ar = [| 1; 2; 3; 4 |] ;;
ar.(2) <- 10 ;;
ar

val ar : int array = [|1; 2; 3; 4|]


- : unit = ()


- : int array = [|1; 2; 10; 4|]


Notice how we access an element in the array with the operator `.()` and the operator `<-`, indexes start at 0. You notice the modifying operation returned `unit = ()`, that is the return value when you return nothing.

### Mutable records

You can have mutable records, the mutation operation will return `unit = ()`

In [11]:
type running_sum = {
    mutable sum: float;
    mutable sum_sq: float;
    mutable samples: int;
} ;;

(* This creates an empty record, like a ctor *)
let create () = {sum = 0.; sum_sq = 0.; samples = 0}

let update rsum x =
  rsum.samples <- rsum.samples + 1;
  rsum.sum <- rsum.sum +. x;
  rsum.sum_sq <- rsum.sum_sq +. x *. x;
;;

type running_sum = {
  mutable sum : float;
  mutable sum_sq : float;
  mutable samples : int;
}


val create : unit -> running_sum = <fun>


val update : running_sum -> float -> unit = <fun>


Notice how we mutate a _mutable_ field with `<-`.

### Mutable variables

When you assign a value with `let` that is an immutable value. You can create a single mutable field with `let` too using a special type named `ref` which is defined simply as a record of type `{contents : 'a}`

In [20]:
let x = {contents = 10};;
x.contents <- x.contents - 3 ;;
x.contents

val x : int Core.ref = {contents = 10}


- : unit = ()


- : int = 7


The type `ref` have a specific operator to create them, `ref`, `!` to get its content and `:=` to assign its content:

In [22]:
let x = ref 10 ;;
x := !x - 7;;
x

val x : int Core.ref = {contents = 10}


- : unit = ()


- : int Core.ref = {contents = 3}


You can see many nice things about the OCaml standard library, like, for example, the book explained how to implement ref and its operators from scratch:

In [24]:
type 'a ref = { mutable contents: 'a } ;;
let ref r = { contents = r } ;;
let (!) r = r.contents ;;
let (:=) r x = r.contents <- x

type 'a ref = { mutable contents : 'a; }


val ref : 'a -> 'a ref = <fun>


val ( ! ) : 'a ref -> 'a = <fun>


val ( := ) : 'a ref -> 'a -> unit = <fun>


Ah the beauty of composable types!

### For loops...

Believe it or not, OCaml has for loops as well, they are useful to handle _mutable structures_ like arrays

In [30]:
let sum_array array x =
  let length = Array.length array in
  for i = 0 to length - 1 do
    array.(i) <- array.(i) + x
  done
;;

sum_array  [|1; 2; 3|] 2 ;;

val sum_array : int Core.Array.t -> int -> unit = <fun>


- : unit = ()
